# CMAP EMAT Univariate Sensitivity Tests

In [1]:
import emat
import os
import pandas as pd
import numpy as np
import gzip
from emat.util.show_dir import show_dir, show_file_contents

In [2]:
import logging
from emat.util.loggers import log_to_stderr
log = log_to_stderr(logging.DEBUG)

## Connecting to the Model

The interface for this model is located in the `cmap_emat.py`
module, which we will import into this notebook.  This file is extensively
documented in comments, and is a great starting point for new users
who want to edit this interface.

In [3]:
import cmap_emat

Within this module, you will find a definition for the 
`CMAP_EMAT_Model` class.

We initialize an instance of the model interface object.
If you look at the module code, you'll note the `__init__` function
does a number of things, including
loading the scope, and creating a SQLite database to work within.

**NEW**: The `unique_id` defines a copy directory next to the original main
directory.  If you don't provide one, it is a random value.  The entire
model is copied to the new directory before anthing else happens.  You can re-use
the same `unique_id` and not create all-new versions.  But this way you can open
a second notebook, change the value of `unique_id`, and run the model in parallel
with different inputs.

In [4]:
fx = cmap_emat.CMAP_EMAT_Model(unique_id="emat-test-uni-20200815-jpn", db_filename="database-20200812-uni-2.db")

[05:37.81] MainProcess/INFO: CMAP EMAT Model INIT...
[05:38.24] MainProcess/INFO: CMAP EMAT Model INIT complete.


## Single Run Operation for Development and Debugging

Before we take on the task of running this model in exploratory mode, we'll
want to make sure that our interface code is working correctly. To check each
of the components of the interface (setup, run, post-process, load-measures,
and archive), we can run each individually in sequence, and inspect the results
to make sure they are correct.

### setup

This method is the place where the core model *set up* takes place,
including creating or modifying files as necessary to prepare
for a core model run.  When running experiments, this method
is called once for each core model experiment, where each experiment
is defined by a set of particular values for both the exogenous
uncertainties and the policy levers.  These values are passed to
the experiment only here, and not in the `run` method itself.
This facilitates debugging, as the `setup` method can be used 
without the `run` method, as we do here. This allows us to manually
inspect the prepared files and ensure they are correct before
actually running a potentially expensive model.

Each input exogenous uncertainty or policy lever can potentially
be used to manipulate multiple different aspects of the underlying
core model.  For example, a policy lever that includes a number of
discrete future network "build" options might trigger the replacement
of multiple related network definition files.  Or, a single uncertainty
relating to the cost of fuel might scale both a parameter linked to
the modeled per-mile cost of operating an automobile and the
modeled total cost of fuel used by transit services.

In [ ]:
params = fx.scope.get_parameter_defaults()
params

In [ ]:
# fx.setup(params)

### run

The `run` method is the place where the core model run takes place.
Note that this method takes no arguments; all the input
exogenous uncertainties and policy levers are delivered to the
core model in the `setup` method, which will be executed prior
to calling this method. This facilitates debugging, as the `setup`
method can be used without the `run` method as we did above, allowing
us to manually inspect the prepared files and ensure they
are correct before actually running a potentially expensive model.

In [ ]:
# fx.run() # ready to go

In [ ]:
STOP

### load-measures

The `load_measures` method is the place to actually reach into
files in the core model's run results and extract performance
measures, returning a dictionary of key-value pairs for the 
various performance measures. It takes an optional list giving a 
subset of performance measures to load, and like the `post_process` 
method also can be pointed at an archive location instead of loading 
measures from the local working directory (which is the default).
The `load_measures` method should not do any post-processing
of results (i.e. it should read from but not write to the model
outputs directory).

In [ ]:
#fx.load_measures()

### archive

The `archive` method copies the relevant model output files to an archive location for 
longer term storage.  The particular archive location is based on the experiment id
for a particular experiment, and can be customized if desired by overloading the 
`get_experiment_archive_path` method.  This customization is not done in this demo,
so the default location is used.

In [ ]:
#fx.get_experiment_archive_path(parameters=params)

Actually running the `archive` method should copy any relevant output files
from the `model_path` of the current active model into a subdirectory of `archive_path`.

In [ ]:
#fx.archive(params)

It is permissible, but not required, to simply copy the entire contents of the 
former to the latter, as is done in this example. However, if the current active model
directory has a lot of boilerplate files that don't change with the inputs, or
if it becomes full of intermediate or temporary files that definitely will never
be used to compute performance measures, it can be advisable to selectively copy
only relevant files. In that case, those files and whatever related sub-directory
tree structure exists in the current active model should be replicated within the
experiments archive directory.

In [ ]:
STOP

## Multiprocessing for Running Multiple Experiments



### Automatic Multiprocessing for Running Multiple Experiments

The examples above are all essentially single-process demonstrations of using TMIP-EMAT to run core model
experiments, either by running all in one single process, or by having a user manually instantiate a number 
of single processes.  If your core model itself is multi-threaded or otherwise is designed to make 
full use of your multi-core CPU, or if a single core model run will otherwise max out some
computational resource (e.g. RAM, disk space) then single process operation should be sufficient.

If, on the other hand, your model is such that you can run multiple independent instances of
the model side-by-side on the same machine, but you don't want to manage the process of manually, 
then you could benefit from a multiprocessing approach that uses the `dask.distributed` library.  To
demonstrate this, we'll create yet another small design of experiments to run.

In [ ]:
design3 = fx.design_experiments(design_name='uni', random_seed=3, sampler='uni')
design3

In [5]:
uni = fx.read_experiment_parameters(design_name='uni')
uni

,global_loops,highway_cap,telecommuting,fuel_cost,fuel_economy,vmt_charge,land_use,vot_sensitivity,park_price,transit_fares,expressway_toll
experiment,,,,,,,,,,,
1,4,1.0,0.1,3.0,0.7,0.0,base,0.8,0.5,1.0,0.1
2,4,0.9,0.1,3.0,0.7,0.0,base,0.8,0.5,1.0,0.1
3,4,1.5,0.1,3.0,0.7,0.0,base,0.8,0.5,1.0,0.1
4,4,1.0,0.0,3.0,0.7,0.0,base,0.8,0.5,1.0,0.1
5,4,1.0,0.5,3.0,0.7,0.0,base,0.8,0.5,1.0,0.1
6,4,1.0,0.1,2.5,0.7,0.0,base,0.8,0.5,1.0,0.1
7,4,1.0,0.1,6.0,0.7,0.0,base,0.8,0.5,1.0,0.1
8,4,1.0,0.1,3.0,0.6,0.0,base,0.8,0.5,1.0,0.1
9,4,1.0,0.1,3.0,0.8,0.0,base,0.8,0.5,1.0,0.1


In [6]:
uni_go = uni.loc[[7,9,10,11,16,]]
uni_go

,global_loops,highway_cap,telecommuting,fuel_cost,fuel_economy,vmt_charge,land_use,vot_sensitivity,park_price,transit_fares,expressway_toll
experiment,,,,,,,,,,,
7,4,1.0,0.1,6.0,0.7,0.0,base,0.8,0.5,1.0,0.1
9,4,1.0,0.1,3.0,0.8,0.0,base,0.8,0.5,1.0,0.1
10,4,1.0,0.1,3.0,0.7,0.1,base,0.8,0.5,1.0,0.1
11,4,1.0,0.1,3.0,0.7,0.0,alt1,0.8,0.5,1.0,0.1
16,4,1.0,0.1,3.0,0.7,0.0,base,0.8,0.5,2.0,0.1


The demo module is set up to facilitate distributed multiprocessing. During the `setup`
step, the code detects if it is being run in a distributed "worker" environment instead of
in a normal Python environment.  If the "worker" environment is detected, then a copy
of the entire files-based model is made into the worker's local workspace, and the model
is run there instead of in the master workspace.  This allows each worker to edit the files
independently and simultaneously, without disturbing other parallel workers.

With this small modification, we are ready to run this demo model in parallel subprocesses.
to do, we simply import the `get_client` function, and use that for the `evaluator` argument
in the `run_experiments` method.

In [7]:
from emat.util.distributed import get_client # for multi-process operation
client=get_client(n_workers=8)
fx.run_experiments(design=uni_go, evaluator=client)

[06:16.59] MainProcess/INFO: performing 5 scenarios/policies * 1 model(s) = 5 experiments
[06:16.62] MainProcess/DEBUG: evaluating experiments asynchronously
[06:16.62] MainProcess/DEBUG: receiving experiments asynchronously
[39:40:02.63] MainProcess/DEBUG: storing scenario 0 for policy Incognito0 on model CMAPTripBasedModel
[39:40:02.63] MainProcess/DEBUG: 1 cases completed
[39:40:02.63] MainProcess/INFO: 1 cases completed
[39:40:02.64] MainProcess/DEBUG: storing Chicago_Expressway_VMT
[39:40:02.64] MainProcess/DEBUG: storing Chicago_Arterial_VMT
[39:40:02.64] MainProcess/DEBUG: storing Chicago_RampToll_VMT
[39:40:02.64] MainProcess/DEBUG: storing Chicago_Centroid_VMT
[39:40:02.65] MainProcess/DEBUG: storing Chicago_Total_District_VMT
[39:40:02.65] MainProcess/DEBUG: storing Cook_balance_Expressway_VMT
[39:40:02.65] MainProcess/DEBUG: storing Cook_balance_Arterial_VMT
[39:40:02.65] MainProcess/DEBUG: storing Cook_balance_RampToll_VMT
[39:40:02.65] MainProcess/DEBUG: storing Cook_balan

[39:40:02.76] MainProcess/DEBUG: storing NON_ATTAINMENT_AREA_Vehicle_Class_VMT_B_Plate_Truck_VMT
[39:40:02.76] MainProcess/DEBUG: storing NON_ATTAINMENT_AREA_Vehicle_Class_VMT_Light_Truck_VMT
[39:40:02.77] MainProcess/DEBUG: storing NON_ATTAINMENT_AREA_Vehicle_Class_VMT_Medium_Truck_VMT
[39:40:02.77] MainProcess/DEBUG: storing NON_ATTAINMENT_AREA_Vehicle_Class_VMT_Heavy_Truck_VMT
[39:40:02.77] MainProcess/DEBUG: storing NON_ATTAINMENT_AREA_Vehicle_Class_VMT_Bus_VMT
[39:40:02.77] MainProcess/DEBUG: storing NON_ATTAINMENT_AREA_Vehicle_Class_VMT_All_VMT
[39:40:02.77] MainProcess/DEBUG: storing NonWork_Pace_Board
[39:40:02.77] MainProcess/DEBUG: storing NonWork_Pace_PMT
[39:40:02.77] MainProcess/DEBUG: storing NonWork_Pace_PHT
[39:40:02.77] MainProcess/DEBUG: storing NonWork_CTA_Bus_Board
[39:40:02.77] MainProcess/DEBUG: storing NonWork_CTA_Bus_PMT
[39:40:02.77] MainProcess/DEBUG: storing NonWork_CTA_Bus_PHT
[39:40:02.78] MainProcess/DEBUG: storing NonWork_CTA_Rail_Board
[39:40:02.78] Main

[39:40:02.89] MainProcess/DEBUG: storing EJ_Person_Trips_NHej_Aut_Per_Trips
[39:40:02.89] MainProcess/DEBUG: storing EJ_Person_Trips_NHej_Tra_Per_Trips
[39:40:02.89] MainProcess/DEBUG: storing EJ_Transit_Share_HWej_Tran_Shr
[39:40:02.89] MainProcess/DEBUG: storing EJ_Transit_Share_HOej_Tran_Shr
[39:40:02.89] MainProcess/DEBUG: storing EJ_Transit_Share_NHej_Tran_Shr
[39:40:02.90] MainProcess/DEBUG: storing EJ_Transit_Share_Totej_Tran_Shr
[39:40:02.90] MainProcess/DEBUG: storing EJ_Average_Trip_Time_HWej_Aut_Avg_Min
[39:40:02.90] MainProcess/DEBUG: storing EJ_Average_Trip_Time_HOej_Aut_Avg_Min
[39:40:02.90] MainProcess/DEBUG: storing EJ_Average_Trip_Time_NHej_Aut_Avg_Min
[39:40:02.90] MainProcess/DEBUG: storing Average_EJ_TRANSIT_Trip_Time_HWej_Trn_Avg_Min
[39:40:02.90] MainProcess/DEBUG: storing Average_EJ_TRANSIT_Trip_Time_HOej_Trn_Avg_Min
[39:40:02.90] MainProcess/DEBUG: storing Average_EJ_TRANSIT_Trip_Time_NHej_trn_Avg_Min
[39:40:02.90] MainProcess/DEBUG: storing mf44_amtime_311_to_2

[39:40:03.00] MainProcess/DEBUG: storing mf47_mddist_384_to_125
[39:40:03.01] MainProcess/DEBUG: storing mf47_mddist_384_to_511
[39:40:03.01] MainProcess/DEBUG: storing mf47_mddist_384_to_2049
[39:40:03.01] MainProcess/DEBUG: storing mf47_mddist_623_to_24
[39:40:03.01] MainProcess/DEBUG: storing mf47_mddist_623_to_125
[39:40:03.01] MainProcess/DEBUG: storing mf47_mddist_623_to_511
[39:40:03.02] MainProcess/DEBUG: storing mf47_mddist_623_to_2049
[39:40:03.02] MainProcess/DEBUG: storing mf47_mddist_1636_to_24
[39:40:03.02] MainProcess/DEBUG: storing mf47_mddist_1636_to_125
[39:40:03.02] MainProcess/DEBUG: storing mf47_mddist_1636_to_511
[39:40:03.02] MainProcess/DEBUG: storing mf47_mddist_1636_to_2049
[39:40:03.02] MainProcess/DEBUG: storing mf47_mddist_2004_to_24
[39:40:03.02] MainProcess/DEBUG: storing mf47_mddist_2004_to_125
[39:40:03.02] MainProcess/DEBUG: storing mf47_mddist_2004_to_511
[39:40:03.02] MainProcess/DEBUG: storing mf47_mddist_2004_to_2049
[39:40:03.02] MainProcess/DEBUG

[39:40:03.11] MainProcess/DEBUG: storing DuPage_Heavy_Trucks_Arterial_VMT
[39:40:03.11] MainProcess/DEBUG: storing DuPage_Heavy_Trucks_RampToll_VMT
[39:40:03.12] MainProcess/DEBUG: storing DuPage_Heavy_Trucks_Centroid_VMT
[39:40:03.12] MainProcess/DEBUG: storing DuPage_Heavy_Trucks_Total_District_VMT
[39:40:03.12] MainProcess/DEBUG: storing DuPage_Total_VHT_Expressway_VHT
[39:40:03.12] MainProcess/DEBUG: storing DuPage_Total_VHT_Arterial_VHT
[39:40:03.12] MainProcess/DEBUG: storing DuPage_Total_VHT_RampToll_VHT
[39:40:03.12] MainProcess/DEBUG: storing DuPage_Total_VHT_Centroid_VHT
[39:40:03.12] MainProcess/DEBUG: storing DuPage_Total_VHT_Total_District_VHT
[39:40:03.12] MainProcess/DEBUG: storing Kane_Autos_Expressway_VMT
[39:40:03.12] MainProcess/DEBUG: storing Kane_Autos_Arterial_VMT
[39:40:03.12] MainProcess/DEBUG: storing Kane_Autos_RampToll_VMT
[39:40:03.12] MainProcess/DEBUG: storing Kane_Autos_Centroid_VMT
[39:40:03.13] MainProcess/DEBUG: storing Kane_Autos_Total_District_VMT
[3

...

[40:12:07.06] MainProcess/DEBUG: storing mf45_amdist_623_to_2049
[40:12:07.06] MainProcess/DEBUG: storing mf45_amdist_1636_to_24
[40:12:07.06] MainProcess/DEBUG: storing mf45_amdist_1636_to_125
[40:12:07.06] MainProcess/DEBUG: storing mf45_amdist_1636_to_511
[40:12:07.06] MainProcess/DEBUG: storing mf45_amdist_1636_to_2049
[40:12:07.06] MainProcess/DEBUG: storing mf45_amdist_2004_to_24
[40:12:07.06] MainProcess/DEBUG: storing mf45_amdist_2004_to_125
[40:12:07.06] MainProcess/DEBUG: storing mf45_amdist_2004_to_511
[40:12:07.07] MainProcess/DEBUG: storing mf45_amdist_2004_to_2049
[40:12:07.07] MainProcess/DEBUG: storing mf45_amdist_2203_to_24
[40:12:07.07] MainProcess/DEBUG: storing mf45_amdist_2203_to_125
[40:12:07.07] MainProcess/DEBUG: storing mf45_amdist_2203_to_511
[40:12:07.07] MainProcess/DEBUG: storing mf45_amdist_2203_to_2049
[40:12:07.07] MainProcess/DEBUG: storing mf45_amdist_2290_to_24
[40:12:07.07] MainProcess/DEBUG: storing mf45_amdist_2290_to_125
[40:12:07.07] MainProcess/

[40:12:07.18] MainProcess/DEBUG: storing Chicago_Total_VHT_Arterial_VHT
[40:12:07.18] MainProcess/DEBUG: storing Chicago_Total_VHT_RampToll_VHT
[40:12:07.18] MainProcess/DEBUG: storing Chicago_Total_VHT_Centroid_VHT
[40:12:07.18] MainProcess/DEBUG: storing Chicago_Total_VHT_Total_District_VHT
[40:12:07.19] MainProcess/DEBUG: storing Cook_balance_Autos_Expressway_VMT
[40:12:07.19] MainProcess/DEBUG: storing Cook_balance_Autos_Arterial_VMT
[40:12:07.19] MainProcess/DEBUG: storing Cook_balance_Autos_RampToll_VMT
[40:12:07.19] MainProcess/DEBUG: storing Cook_balance_Autos_Centroid_VMT
[40:12:07.19] MainProcess/DEBUG: storing Cook_balance_Autos_Total_District_VMT
[40:12:07.19] MainProcess/DEBUG: storing Cook_balance_B-plate_Trucks_Expressway_VMT
[40:12:07.19] MainProcess/DEBUG: storing Cook_balance_B-plate_Trucks_Arterial_VMT
[40:12:07.19] MainProcess/DEBUG: storing Cook_balance_B-plate_Trucks_RampToll_VMT
[40:12:07.20] MainProcess/DEBUG: storing Cook_balance_B-plate_Trucks_Centroid_VMT
[40

[40:12:07.31] MainProcess/DEBUG: storing Kendall_Medium_Trucks_Expressway_VMT
[40:12:07.31] MainProcess/DEBUG: storing Kendall_Medium_Trucks_Arterial_VMT
[40:12:07.31] MainProcess/DEBUG: storing Kendall_Medium_Trucks_RampToll_VMT
[40:12:07.31] MainProcess/DEBUG: storing Kendall_Medium_Trucks_Centroid_VMT
[40:12:07.31] MainProcess/DEBUG: storing Kendall_Medium_Trucks_Total_District_VMT
[40:12:07.31] MainProcess/DEBUG: storing Kendall_Heavy_Trucks_Expressway_VMT
[40:12:07.31] MainProcess/DEBUG: storing Kendall_Heavy_Trucks_Arterial_VMT
[40:12:07.31] MainProcess/DEBUG: storing Kendall_Heavy_Trucks_RampToll_VMT
[40:12:07.31] MainProcess/DEBUG: storing Kendall_Heavy_Trucks_Centroid_VMT
[40:12:07.31] MainProcess/DEBUG: storing Kendall_Heavy_Trucks_Total_District_VMT
[40:12:07.31] MainProcess/DEBUG: storing Kendall_Total_VHT_Expressway_VHT
[40:12:07.32] MainProcess/DEBUG: storing Kendall_Total_VHT_Arterial_VHT
[40:12:07.32] MainProcess/DEBUG: storing Kendall_Total_VHT_RampToll_VHT
[40:12:07.3

[40:12:07.41] MainProcess/DEBUG: storing Illinois_balance_B-plate_Trucks_Arterial_VMT
[40:12:07.41] MainProcess/DEBUG: storing Illinois_balance_B-plate_Trucks_RampToll_VMT
[40:12:07.41] MainProcess/DEBUG: storing Illinois_balance_B-plate_Trucks_Centroid_VMT
[40:12:07.41] MainProcess/DEBUG: storing Illinois_balance_B-plate_Trucks_Total_District_VMT
[40:12:07.41] MainProcess/DEBUG: storing Illinois_balance_Light_Trucks_Expressway_VMT
[40:12:07.41] MainProcess/DEBUG: storing Illinois_balance_Light_Trucks_Arterial_VMT
[40:12:07.41] MainProcess/DEBUG: storing Illinois_balance_Light_Trucks_RampToll_VMT
[40:12:07.41] MainProcess/DEBUG: storing Illinois_balance_Light_Trucks_Centroid_VMT
[40:12:07.41] MainProcess/DEBUG: storing Illinois_balance_Light_Trucks_Total_District_VMT
[40:12:07.41] MainProcess/DEBUG: storing Illinois_balance_Medium_Trucks_Expressway_VMT
[40:12:07.41] MainProcess/DEBUG: storing Illinois_balance_Medium_Trucks_Arterial_VMT
[40:12:07.42] MainProcess/DEBUG: storing Illinois_

KeyboardInterrupt: 

In [8]:
client

Client Scheduler: tcp://127.0.0.1:58759 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 274.51 GB


In [9]:
client.restart()

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing


Client Scheduler: tcp://127.0.0.1:58759 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 274.51 GB
